In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import  ChatPromptTemplate, MessagesPlaceholder

llm  = ChatOpenAI(model_name="gpt-4",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    #memory_key="chat_history", #기본 키는 history다 
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI taking to a human." ),
        MessagesPlaceholder(variable_name="history"), #message형식일 때 히스토리 제공방법
        ("human", "{question}"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]
#RunnablePassthrough : 프롬프트가 완성되기 전에 함수를 실행시켜줌
#그리고 assign 메소드를 통해 어느 값이든 할당할 수 있음
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context({"input": question},{"output": result.content}) 
    print(result)



In [18]:
invoke_chain("My name is jonas")

content='Nice to meet you, Jonas! How can I assist you today?'


In [19]:
invoke_chain("I live in Seoul")

content="That's great, Jonas! Seoul is a vibrant city with a rich history and culture. How can I assist you further? Are you looking for recommendations on places to visit or need information about something specific in Seoul?"


In [20]:
invoke_chain(question="What is my name?")

content='Your name is Jonas.'
